## Importação das bibliotecas

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob

## Setup

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

board_size = (7, 6)
# resolution = (640, 480)
img_points = []
obj_points = []

h, w = board_size
objp = np.zeros((h*w, 3), np.float32)
objp[:, :2] = np.mgrid[0:h, 0:w].T.reshape(-1, 2)

images = glob.glob('data/*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    ret, corners = cv.findChessboardCorners(gray, (h, w), None)
    if ret == True:
        obj_points.append(objp)
        
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        img_points.append(corners2)

        cv.drawChessboardCorners(img, (h, w), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(0)
        
cv.destroyAllWindows()

## Executa a calibração da câmera

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

## Remove a distorção das imagens capturadas pela câmera

In [ ]:
original_images = []
undistorted_images = []

for fname in images:
    img = cv.imread(fname)
    original_images.append(img)
    
    h, w = img.shape[:2]
    new_mtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    
    new_img = cv.undistort(img, mtx, dist, None, new_mtx)
    
    # crop
    x, y, w, h, = roi
    new_img = new_img[y:y+h, x:x+w]
    undistorted_images.append(new_img)

## Configura e plota as imagens ajustadas

In [ ]:
fig, ax = plt.subplots(nrows=15, ncols=2, figsize=(10, 30))

for i in range(15):
# for i in range(len(images):
    row = i
    col = 0
    
    # Plota a imagem original na coluna 0 da linha atual
    ax[row, col].imshow(original_images[i][:, :, ::-1])
    ax[row, col].axis('off')
    ax[row, col].set_title(f'Imagem {row+1} original')

    # Plota a imagem ajustada na coluna 1 da linha atual
    ax[row, col+1].imshow(undistorted_images[i][:, :, ::-1])
    ax[row, col+1].axis('off')
    ax[row, col+1].set_title(f'Imagem {row+1} ajustada')